## Runnable Sequence

RunableSequence 是 LangChain 中一个强大的工具，它提供了一种将多个 Runnable 对象组合成链式流程的能力。这种链式流程可以将大模型与外部世界连接起来，从而发挥大模型更强大的语言处理能力。

**核心概念**
RunableSequence 的核心思想是将多个 Runnable 对象像流水线一样连接起来，前一个 Runnable 对象的输出作为后一个 Runnable 对象的输入。这样，我们就可以构建复杂的流程，例如：

1. 从外部知识来源获取信息： 例如，从搜索引擎、数据库或 API 中检索相关信息。
1. 将信息传递给大模型： 将检索到的信息作为上下文提供给大模型。
1. 大模型生成文本： 大模型根据上下文生成更专业、更准确的文本。
1. 对文本进行处理： 例如，对文本进行过滤、排序或格式化。
1. 将结果返回给用户： 将处理后的文本呈现给用户。

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain.chains import MultiRetrievalQAChain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from pprint import pprint

from chat_model_client import get_model

## 1. 检索多个数据源

In [2]:
embeddings = OllamaEmbeddings(model = "llama2-chinese")
## 中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力
loader = WebBaseLoader("https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a")
retriever1 = Chroma.from_documents(loader.load(), embedding=embeddings).as_retriever(search_kwargs={"k": 1})

## 算一算马拉松热背后的经济账
loader = WebBaseLoader("https://cn.chinadaily.com.cn/a/202412/04/WS67502215a310b59111da71ab.html")
retriever2 = Chroma.from_documents(loader.load(), embedding=embeddings).as_retriever(search_kwargs={"k": 1})

## 强冷空气将影响中东部地区
loader = WebBaseLoader("https://www.cma.gov.cn/2011xwzx/2011xqxxw/2011xzytq/202502/t20250205_6838360.html")
retriever3 = Chroma.from_documents(loader.load(), embedding=embeddings).as_retriever(search_kwargs={"k": 1})


retriever_infos = [
    {
        "name": "中国马拉松产业观察",
        "description": "中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力",
        "retriever": retriever1
    },
    {
        "name": "马拉松热背后的经济账",
        "description": "马拉松热背后的经济账",
        "retriever": retriever2
    },
    {
        "name": "影响中东部地区",
        "description": "强冷空气将影响中东部地区",
        "retriever": retriever3
    }
]

llm = get_model("llama")
print(llm)
chain = MultiRetrievalQAChain.from_retrievers(llm, retriever_infos, default_chain_llm = llm)

print("命中", chain.invoke("中东部地区的天气怎么样？"))
print("命中", chain.invoke("马拉松经济价值"))
print("未命中", chain.invoke("什么手机性价比更高？"))

model='llama2-chinese' temperature=0.0 base_url='http://localhost:11434'


/Users/shaon/anaconda3/envs/wrenv3.10/lib/python3.10/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


命中 {'input': '中东部地区的天气怎么样？', 'query': '中东部地区的天气怎么样？', 'result': '\n中东部地区的天气非常多样化，因为这个地区包含了许多不同国家和地区。以下是一些特定国家或地区的天气情况：\n\n1. 阿联酋：阿联酋的天气通常非常炎热，年平均温度在40°C以上。夏季时，最高温度可达到50°C以上，而冬季时，最低温度也不低于10°C。\n\n2. 巴林：巴林的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到45°C以上，而冬季时，最低温度也不低于10°C。\n\n3. 科威特：科威特的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到40°C以上，而冬季时，最低温度也不低于10°C。\n\n4. 阿曼：阿曼的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到40°C以上，而冬季时，最低温度也不低于10°C。\n\n5. 阿拉伯联合酋长国：阿拉伯联合酋长国的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到45°C以上，而冬季时，最低温度也不低于10°C。\n\n6. 伊拉克：伊拉克的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到45°C以上，而冬季时，最低温度也不低于10°C。\n\n7. 约旦：约旦的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到40°C以上，而冬季时，最低温度也不低于10°C。\n\n8. 苏丹：苏丹的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到40°C以上，而冬季时，最低温度也不低于10°C。\n\n9. 埃及：埃及的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到40°C以上，而冬季时，最低温度也不低于10°C。\n\n10. 利比里亚：利比里亚的天气非常炎热，年平均温度在28°C以上。夏季时，最高温度可达到40°C以上，而冬季时，最低温度也不低于10°C。\n'}
命中 {'input': '马拉松经济价值', 'query': '马拉松经济价值', 'result': '\n马拉松经济价值是指一个城市或地区在马拉松赛事上获得的经济效益。这些效益可以来自于各种商业活动，例如销售、广告和购买等。\n\n根据一项研究，马拉松赛事的经济影响非常大。每年的马拉松赛事可以为城市或地区带来数百万

## 2. 先写后读

在 RunnableSequence 中使用大模型、查询语句和外部文档进行文本处理的流程。核心思想是利用大模型生成文本，然后将生成的文本转换为查询语句，再通过 Retriever 对象从外部文档中检索相关信息，最后将检索到的信息注入到流程中，以达到纠错或扩展文本的目的。

In [3]:
from operator import itemgetter

document_prompt = PromptTemplate.from_template("{page_content}")

def _combine_documents(docs, document_prompt = document_prompt, document_separator="\n\n"):
    doc_string = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_string)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
)

template = """
    Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question. in its original language.

    Chat History:
    {chat_history}
    Follow Up Input: {question}
    Standalone question:
"""

# condense_question_prompt = PromptTemplate.from_template(template)
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: x["chat_history"],
    } | PromptTemplate.from_template(template) | llm | StrOutputParser()
}

retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever1,
    "question": lambda x: x["standalone_question"],
}

ANSWER_PROMPT = """
    Answer the questions based on the context provided.

    Context:
    {context}
    Question:
    {question}

    answer:
"""

final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}


answer_question = {
    "answer": final_inputs | PromptTemplate.from_template(ANSWER_PROMPT) | llm,
    "docs": itemgetter("docs"),
}

final_chain = loaded_memory | standalone_question | retrieved_documents | answer_question

final_chain.invoke({"question": "马拉松经济价值"})

{'answer': AIMessage(content='\nThe economic value of a marathon can be significant. According to a report by the International Association of Athletics Federations (IAAF), the global marathon industry generated $1.3 billion in revenue in 2019, with the majority coming from sponsorships and broadcasting rights. Additionally, the marathon industry also creates jobs and stimulates local economies through tourism and other related activities.\n', additional_kwargs={}, response_metadata={'model': 'llama2-chinese', 'created_at': '2025-02-07T12:27:56.80214Z', 'done': True, 'done_reason': 'stop', 'total_duration': 82977181834, 'load_duration': 18577417, 'prompt_eval_count': 2048, 'prompt_eval_duration': 61595000000, 'eval_count': 88, 'eval_duration': 21278000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-aa418d4e-51b4-4758-80f4-627650ff97e5-0', usage_metadata={'input_tokens': 2048, 'output_tokens': 88, 'total_tokens': 2136}),
 'docs': [Document(i

In [8]:
pprint(final_chain.get_graph().print_ascii())

                     +-----------------------------+              
                     | Parallel<chat_history>Input |              
                     +-----------------------------+              
                            **              ***                   
                         ***                   **                 
                       **                        ***              
        +-----------------------+                   **            
        | load_memory_variables |                    *            
        +-----------------------+                    *            
                     *                               *            
                     *                               *            
                     *                               *            
                +--------+                   +-------------+      
                | Lambda |                   | Passthrough |      
                +--------+**                 +-------------+  